In [46]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
pip install HTMLParser

In [4]:
pip install beautifulsoup4

In [11]:
import json

path = "/content/drive/My Drive/5001_Group/"
js = open(path + "Sarcasm_Headlines_Dataset.json")
dataset = []
for line in js.readlines():
  js_line = json.loads(line)
  dataset.append(js_line)
  
headlines = []
labels = []
links = []
for item in dataset:
  headlines.append(item["headline"])
  labels.append(int(item["is_sarcastic"]))
  links.append(item["article_link"])

In [12]:
import requests
import csv
import socket
import random
import http.client
import time
from bs4 import BeautifulSoup
import tqdm

def get_content(url , data = None):
    header={
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, sdch',
        'Accept-Language': 'zh-CN,zh;q=0.8',
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.235'
    }
    timeout = random.choice(range(80, 180))
    i = 0
    while i < 2:
        try:
            rep = requests.get(url,headers = header,timeout = timeout)
            rep.encoding = 'utf-8'
            break
        except socket.timeout as e:
            # print( '3:', e)
            time.sleep(random.random())

        except socket.error as e:
            # print( '4:', e)
            time.sleep(random.random())

        except http.client.BadStatusLine as e:
            # print( '5:', e)
            time.sleep(random.random())

        except http.client.IncompleteRead as e:
            # print( '6:', e)
            time.sleep(random.random())
        i = i + 1
    if i >= 2:
      return False
    else:
      return rep.text

def get_data(html_text, source):
    s = ""
    bs = BeautifulSoup(html_text, "html.parser")  # 创建BeautifulSoup对象
    body = bs.body # 获取body部分
    if source == "h":
      data = body.find_all(class_="primary-cli cli cli-text ")
      for i in data:
        if i.p:
          s= s + i.p.text + " "
    else:
      data = body.find_all(class_="sc-77igqf-0 bOfvBY")
      for i in data:
        if i.text:
          s = s+ i.text + " "
    return s.replace("\n", " ")

bad_link = []
articals = []
tick = 0
for idx, link in enumerate(tqdm.tqdm(links)):
  if "huff" in link:
    scource = "h"
  else:
    scource = "o"
  result = get_content(link, scource)
  if result:
    articals.append(get_data(result, scource))
  else:
    articals.append(" ")
    bad_link.append(link)
  tick = tick + 1

100%|██████████| 26709/26709 [5:30:09<00:00,  1.35it/s]


In [13]:
print(len(labels))
print(len(articals))

26709
26709


In [23]:
# path = "/content/drive/My Drive/5001_Group/artical_fold/"
# for i, label in enumerate(tqdm.tqdm(labels)):
#   path_out = path + "artical_" + str(i) + ".txt"
#   f =  open(path_out, "w")
#   f.write(articals[i])
#   f.close()

100%|██████████| 26709/26709 [02:14<00:00, 199.19it/s]


In [48]:
path = "/content/articals_scource.txt"
with open (path, "w") as f:
  for i, label in enumerate(tqdm.tqdm(labels)):
    f.write(articals[i].replace("\n", " ").replace("\r", " ") + "\n")
f.close()

100%|██████████| 26709/26709 [00:00<00:00, 148449.15it/s]


In [40]:
lines = open("/content/drive/My Drive/5001_Group/articals_scource.txt", "r").readlines()
print(len(lines))

26709


In [37]:
# with open("/content/drive/ My Drive/5001_Group/bad_link.txt", "w") as f:
#   for link in bad_link:
#     f.write(link + "\n")
# f.close()

In [25]:
print(len(bad_link) / len(links))

0.021865288853944363
